# Aula 7: Sobre query eu QUERY MAIS!

Documentação: https://www.postgresql.org/docs/9.1/functions.html

In [2]:
import pandas as pd 
from sqlalchemy import create_engine

USER = "postgres"
SENHA = "senha"
HOST = "localhost"
DATABASE = "postgres"

engine = create_engine(f"postgresql+psycopg2://{USER}:{SENHA}@{HOST}/{DATABASE}")

### CASE WHEN ...

Temos um if then else em SQL, para fazer comparações e preencher valores;

In [3]:
# CASE 
#     WHEN coluna = alguma_coisa THEN novo_valor
#     ELSE outro_valor
# END AS nome_nova_coluna

O código é bastante literal, WHEN (QUANDO) o valor da coluna for igual, maior, menor (algum comparativo) a alguma coisa, THEN (ENTÃO) assume novo_valor. ELSE (CASO CONTRÁRIO) assume outro_valor. 

Ao final, (END) atribuímos um nome a coluna.

Vale a atenção:
- Pode ser feito qualquer comparação após o WHEN: =, !=, <>, <, <=, >, >=;
- THEN vai receber o novo valor que você quer, sempre use aspas simples se não for número;
- O valor em ELSE não é obrigatório, é possível fazer apenas com CASE WHEN __ THEN __ END;
- É possível fazer várias regras de WHEN __ THEN __.


In [4]:
# Na query a seguir, vou criar a coluna "teste";
# Ela atribui o valor 'show' quando a coluna estado é 'PR';
# Caso contrário, a coluna é preenchida com 'show também'.

query = """
SELECT 
    id_cidade, 
    cidade, 
    estado,
    CASE 
        WHEN estado = 'PR' AND cidade = 'curitiba' THEN 'show'
        WHEN estado = 'SP' THEN 'alegria' 
        ELSE 'show também'
    END AS teste
FROM ada.cidades;
"""

df = pd.read_sql_query(query, con = engine)
df

,id_cidade,cidade,estado,teste
0,1,curitiba,PR,show
1,2,sao paulo,SP,alegria
2,3,salvador,BA,show também
3,4,jequié,BA,show também
4,5,tiradentes,mg,show também
5,6,uberlandia,MG,show também
6,7,congonhas,MG,show também
7,8,porto velho,RO,show também
8,9,jaru,RO,show também
9,10,alto paraíso,RO,show também


In [5]:
# Na query a seguir, vou criar a coluna "teste";
# Ela atribui o valor 'TOP' quando a coluna estado é menor que 7;
# Caso contrário, ela não faz nada

query = """
SELECT 
    id_cidade, 
    cidade, 
    estado,
    CASE 
        WHEN id_cidade < 7 THEN 'TOP'
    END AS teste
FROM ada.cidades;
"""

df = pd.read_sql_query(query, con = engine)
df

,id_cidade,cidade,estado,teste
0,1,curitiba,PR,TOP
1,2,sao paulo,SP,TOP
2,3,salvador,BA,TOP
3,4,jequié,BA,TOP
4,5,tiradentes,mg,TOP
5,6,uberlandia,MG,TOP
6,7,congonhas,MG,None
7,8,porto velho,RO,None
8,9,jaru,RO,None
9,10,alto paraíso,RO,None


### SUBQUERYS 

Se uma query representa uma tabela e o FROM recebe uma tabela..., posso substituir uma tabela por uma query?

- Entendo que pode não fazer muito sentido de primeira, mas conforme querys vão atendendo a regras de negócios mais complexas, mais este processo se torna necessário;
- Basicamente, ao invés de fazer "FROM tabela" é feito um "FROM (query) as nome_tabela";
- Atenção ao "as nome_tabela" ao final, é necessário nomear esta tabela provisória;
- Quando as subquerys começam a ficar muito complexas sugem nossas VIEWS.

In [6]:
# SELECT 
#     coluna_1, 
#     coluna_2 
# FROM (
#     SELECT 
#         coluna_1, 
#         coluna_2,
#         coluna_3
#     FROM outra_tabela
# ) AS nome_da_minha_nova_tabela;

In [7]:
# Começamos com essa query com a soma dos valores dos cursos

query = """
SELECT 
    c.curso,
    SUM(c.valor_curso) AS acumulado
FROM ada.transacoes t 
    LEFT JOIN ada.cursos c ON t.id_curso = c.id_curso
GROUP BY 
    c.curso
"""

df = pd.read_sql_query(query, con = engine)
df

,curso,acumulado
0,arquiteto dados,400.0
1,ciencia dados,910.0
2,analise dados,400.0
3,engenharia dados,480.0


In [8]:
# Agora, com essa query acima, queremos o valor médio de todos os cursos
# Logo, vamos fazer uma query na tabela gerada acima 
query = """
SELECT 
    AVG(acumulado) as media_geral
FROM (
    SELECT 
        c.curso,
        SUM(c.valor_curso) AS acumulado
    FROM ada.transacoes t 
        LEFT JOIN ada.cursos c ON t.id_curso = c.id_curso
    GROUP BY 
        c.curso
) AS tabela_de_acumulado
"""

df = pd.read_sql_query(query, con = engine)
df

,media_geral
0,547.5


In [9]:
# Note que o nome da coluna na subquery é o nome que é chamado na query principal
# Ainda, como não temos por quem agrupar, não foi necessário colocar o GROUP BY

# Funções de strings

Documentação: https://www.postgresql.org/docs/9.1/functions-string.html

- LENGTH() -> tamanho da palavra
- UPPER() -> tudo maiúsculo
- LOWER() -> tudo minúsculo 
- INITCAP() -> Primeira Letra Maiúscula 
- CONCAT() -> concatena duas strings
- coluna_1 || coluna_2 -> também concatena strings
- LEFT() -> pega as primeiras letras
- RIGHT() -> pega as últimas letras 
- ...


In [10]:
## Uma query completamente maluca para nos guiar

query = """
SELECT 
cpf, 
primeiro_nome,
ultimo_nome,
endereco,

LENGTH(primeiro_nome) AS "tamanho_nome", -- medir o tamanho das palavras
UPPER(primeiro_nome) AS "NOME", -- deixar tudo maiúsculo
INITCAP(ultimo_nome) AS "Ultimo Nome", -- deixa primeiras letras maiúsculas 
(primeiro_nome || ' ' || ultimo_nome) AS nome_completo, -- concatena os nomes
CONCAT(primeiro_nome, ' ', ultimo_nome) AS nome_completo_2, -- concatena também (mas deixa nulos vazios)
LEFT(endereco, 3) AS tres_primeiras, -- pego as 3 primeiras letras 
RIGHT(endereco, 5) AS cinco_ultimas -- pego as 5 últimas letras

FROM ada.alunos;
"""

df = pd.read_sql_query(query, con = engine)
df.head()

,cpf,primeiro_nome,ultimo_nome,endereco,tamanho_nome,NOME,Ultimo Nome,nome_completo,nome_completo_2,tres_primeiras,cinco_ultimas
0,68740550196,abelar,henrique,Rua da Penha,6,ABELAR,Henrique,abelar henrique,abelar henrique,Rua,Penha
1,34146481132,adriano,gambetta,Rua Henrique Soares,7,ADRIANO,Gambetta,adriano gambetta,adriano gambetta,Rua,oares
2,12728282203,alex,moulin,Rua Leodécio Santiago de Souza,4,ALEX,Moulin,alex moulin,alex moulin,Rua,Souza
3,43764914607,álvaro,fonseca,Rua Pitica,6,ÁLVARO,Fonseca,álvaro fonseca,álvaro fonseca,Rua,itica
4,23228327392,andre,melchior,Rua Camilo Paula,5,ANDRE,Melchior,andre melchior,andre melchior,Rua,Paula


# Funções numéricas

Documentação: https://www.postgresql.org/docs/9.1/functions-math.html

- CAST(coluna AS TYPE) -> converte a tipagem;
- TO_CHAR(coluna, 99D999) -> string com precisão;
- ROUND() -> arredondamento do número;
- DIV() -> divisão;
- POWER() -> potência do número;
- SQRT() -> raíz do número;
- LOG() -> logarítimo com base 10;
- EXP(coluna) -> exponencial do número;
- LN() -> logarítimo natural do número;
- EXP() -> constante de euler;
- PI() -> constante pi;



In [11]:
# query maluca para nos localizarmos
query = """
SELECT 
    curso, 
    valor_curso,
    CAST(valor_curso AS INTEGER), -- conversão para inteiro
    TO_CHAR(valor_curso, '99D999999') AS valor_str, -- conversão para char
    DIV(CAST(valor_curso AS INTEGER), 2) AS metade_valor, -- divisão 
    POWER(valor_curso, 2) AS valor_quadrado, -- potência
    SQRT(valor_curso) AS raiz_valor, -- raiz
    LOG(valor_curso) AS log_base_10, -- logarítimo base 10
    EXP(valor_curso) AS exp_valor_curso, -- exp do número
    LN(valor_curso) AS ln_valor_curso, -- ln do número
    EXP(1.0) as euler_cte, -- constante euler
    PI() as pi_cte -- constante pi

FROM ada.cursos;
"""

df = pd.read_sql_query(query, con = engine)
df

,curso,valor_curso,valor_curso,valor_str,metade_valor,valor_quadrado,raiz_valor,log_base_10,exp_valor_curso,ln_valor_curso,euler_cte,pi_cte
0,engenharia dados,60.0,60,"60,000000",30.0,3600.0,7.745967,1.778151,1.142007e+26,4.094345,2.718282,3.141593
1,ciencia dados,70.0,70,"70,000000",35.0,4900.0,8.366600,1.845098,2.515439e+30,4.248495,2.718282,3.141593
2,analise dados,50.0,50,"50,000000",25.0,2500.0,7.071068,1.698970,5.184706e+21,3.912023,2.718282,3.141593
3,arquiteto dados,80.0,80,"80,000000",40.0,6400.0,8.944272,1.903090,5.540622e+34,4.382027,2.718282,3.141593


# Funções de DATA (ESSA É IMPORTANTE DEMAIS DEMAIS)

Documentação: https://www.postgresql.org/docs/9.1/functions-datetime.html

- DATE_PART('parte', coluna) -> extrai a parte específica de uma data; 
- EXTRACT(parte FROM coluna) -> extrai a parte específica de uma data; 
- AGE(coluna) -> calcula "hoje - coluna";
- NOW() -> pega o timestamp de agora; 
- CURRENT_DATE -> pega a data atual;
- CURRENT_TIME -> pega a hora atual;
- TO_CHAR(data, 'DD/MM/YYYY') -> converte o formato da data em string.

In [12]:
query = """
SELECT 
    primeiro_nome, 
    data_ingresso,
    DATE_PART('day', data_ingresso) AS dia_ingresso,
    DATE_PART('month', data_ingresso) AS mes_ingresso, 
    DATE_PART('year', data_ingresso) AS ano_ingresso,

    EXTRACT(day FROM data_ingresso) AS dia_ingresso_2,
    EXTRACT(month FROM data_ingresso) AS mes_ingresso_2, 
    EXTRACT(year FROM data_ingresso ) AS ano_ingresso_2,

    AGE(data_ingresso) AS tempo_de_ingresso,
    DATE_PART('day', AGE(data_ingresso))::INTEGER AS tempo_de_ingresso_dias,

    NOW() AS timestamp_hoje, 
    NOW()::DATE AS data_hoje,
    CURRENT_DATE AS data_hoje_2,
    CURRENT_TIME AS hora_agora,

    EXTRACT(hour FROM CURRENT_TIME) AS hora_agora,
    EXTRACT(minutes FROM CURRENT_TIME) AS minuto_agora,
    
    DATE_PART('hour', NOW()) AS hora_agora_2, 
    DATE_PART('minutes', NOW()) AS minuto_agora_2,

    TO_CHAR(NOW(), 'DD/MM/YYYY') AS agora_brasil
FROM ada.alunos;
"""

df = pd.read_sql_query(query, con = engine)
df.head()

,primeiro_nome,data_ingresso,dia_ingresso,mes_ingresso,ano_ingresso,dia_ingresso_2,mes_ingresso_2,ano_ingresso_2,tempo_de_ingresso,tempo_de_ingresso_dias,timestamp_hoje,data_hoje,data_hoje_2,hora_agora,hora_agora,minuto_agora,hora_agora_2,minuto_agora_2,agora_brasil
0,abelar,2023-03-05,5.0,3.0,2023.0,5.0,3.0,2023.0,278 days,8,2023-12-14 00:41:44.007284+00:00,2023-12-13,2023-12-13,21:41:44.007284-03:00,21.0,41.0,21.0,41.0,13/12/2023
1,adriano,2023-03-06,6.0,3.0,2023.0,6.0,3.0,2023.0,277 days,7,2023-12-14 00:41:44.007284+00:00,2023-12-13,2023-12-13,21:41:44.007284-03:00,21.0,41.0,21.0,41.0,13/12/2023
2,alex,2023-03-07,7.0,3.0,2023.0,7.0,3.0,2023.0,276 days,6,2023-12-14 00:41:44.007284+00:00,2023-12-13,2023-12-13,21:41:44.007284-03:00,21.0,41.0,21.0,41.0,13/12/2023
3,álvaro,2023-03-08,8.0,3.0,2023.0,8.0,3.0,2023.0,275 days,5,2023-12-14 00:41:44.007284+00:00,2023-12-13,2023-12-13,21:41:44.007284-03:00,21.0,41.0,21.0,41.0,13/12/2023
4,andre,2023-03-09,9.0,3.0,2023.0,9.0,3.0,2023.0,274 days,4,2023-12-14 00:41:44.007284+00:00,2023-12-13,2023-12-13,21:41:44.007284-03:00,21.0,41.0,21.0,41.0,13/12/2023


In [13]:
df = pd.read_sql_query("SELECT * FROM ada_2.alunos;", con = engine)
df.head()

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "ada_2.alunos" does not exist
LINE 1: SELECT * FROM ada_2.alunos;
                      ^

[SQL: SELECT * FROM ada_2.alunos;]
(Background on this error at: https://sqlalche.me/e/14/f405)

### Exerício 1: 

Edite e formate a tabela de alunos para que fique da forma mais bonita e completa que conseguir:

- Crie o nome_completo;
- Formate a data de ingresso;
- Crie colunas de dia, mes, ano;
- Acrescente o ddd;
- formate o telefone;
- Explane o tipo de endereço (rua, avenida, ...);
- Formate o CPF;
- ...

### Exercício 2:

Separe a palavra "dados" de todos os cursos e crie a coluna apenas com os nomes "Engenharia", "Analise", "Arquiteto" e "Cientista", todas com a primeira letra maiúscula

In [25]:
query = """
SELECT 
    INITCAP(SPLIT_PART(curso, ' ', 1))
FROM ada.cursos
"""
# query = """
# SELECT
#     curso,
#     CASE
#         WHEN curso ILIKE '%%engenharia%%' THEN 'Engenharia'
#         WHEN curso ILIKE '%%arquiteto%%' THEN 'Arquiteto'
#         WHEN curso ILIKE '%%ciencia%%' THEN 'Cientista'
# 		WHEN curso ILIKE '%%analise%%' THEN 'Analista'
#         ELSE 'Outro'
#     END AS categoria_curso
# FROM
#     ada.cursos
# WHERE
#     curso ILIKE '%%dados%%';
# """



df = pd.read_sql_query(query, con = engine)
df.head()

,initcap
0,Engenharia
1,Ciencia
2,Analise
3,Arquiteto


### Exercício 3:

Separe nos endereços dos alunos quais são ruas, avenidas e etc

### Exercício 4:

Calcule a quanto tempo o aluno ingressou no curso. Retorne este valor em dias, meses e horas

### Exercício 5:

Encontre quais alunos ingressaram no último ano, nos últimos 11 meses, 10 meses, 9 meses, 8 meses e 7 meses

### Exercício 6:

Formate o cpf dos alunos da forma "ddd.ddd.ddd-dd"

### Exercício 7:

Com as colunas "primeiro_nome" e "curso", crie uma coluna com nome e título do profissional que será formado.

Por exemplo:

`input: primeiro_nome: paulo, curso: engenharia de dados`

`output: Paulo, engenheiro de dados.`

### Exercício 8:

Crie um texto por extenso com algumas informações das informações dos alunos.

Por exemplo:

`input: primeiro_nome: paulo, ultimo_nome: vasques, curso: engenharia de dados, endereco: rua das flores, numero: 10, cidade: curitiba, estado: PR`

`output: Paulo Vasques, morador da Rua das flores, 10, localizada em Curitiba/PR está cursando Engenharia de dados.`